In [1]:
import pandas as pd
import numpy as np
from tqdm import notebook
import ast
import re
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta
import spacy

In [2]:
def translate_rank(rank, translator):
    if rank in translator['(all occupations)'].tolist():
        return translator.index[translator['(all occupations)'] == rank][0]
    else:
        return rank

In [3]:
notebook.tqdm.pandas()
clean = pd.read_csv('../../clean_data.csv')
voc_df = pd.read_csv('../vocop-clustered-new.csv', sep='	')
uuid = []
name = []
for y, z in notebook.tqdm(clean.iterrows()):
    for x in ast.literal_eval(z.namen):
        if x['tussenvoegsel'] != None:
            name.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
            uuid.append(z.uuid)
        elif x['voornaam'] and x['achternaam'] != None:
            name.append(x['voornaam'] + " " + x['achternaam'])
            uuid.append(z.uuid)
name_list = pd.DataFrame(data={'uuid':uuid, 'name':name}, columns=['uuid', 'name'])
name_df = clean.merge(name_list)
rangen = pd.read_excel('../../vocop_rangen.xlsx', index_col=0)
voc_df['dutch_rank'] = [translate_rank(x, rangen) for x in notebook.tqdm(voc_df['rank'].tolist())]

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def fuzzy_search(name, voc, distance):
    names = np.where((voc.fullNameNormalized.apply(fuzz.ratio, args=[name]) >= 90) | 
                     (voc.fullNameOriginal.dropna().apply(fuzz.ratio, args=[name]) >= 90))
    return names[0]

def find_matches(names, voc, distance):
    name_list = {}
    final = []
    for x in notebook.tqdm(names):
        if x in name_list:
            final.append((name_list[x]))
        else:
            result = fuzzy_search(x, voc, distance)
            name_list[x] = result
            final.append(result)
    return final

def get_voc_data(matches, voc):
    if len(matches) != 0:
        data = []
        for index in matches:
            holder = {}
            holder['index'] = index
            holder['name_original'] = voc.iloc[index].fullNameOriginal
            holder['name_normalized'] = voc.iloc[index].fullNameNormalized
            holder['date_out'] = voc.iloc[index].date_begin_service_complete
            holder['date_return'] = voc.iloc[index].date_end_service_complete
            holder['ship_out'] = voc.iloc[index].shipOutward
            holder['ship_return'] = voc.iloc[index].shipReturn
            holder['rank'] = voc.iloc[index]['dutch_rank']
            holder['place_of_origin'] = voc.iloc[index].placeOfOrigin
            data.append(holder)
        return data
    else:
        return 0

def get_notary_matches(row, nlp):
    holder = {}
    if row.data_matches != 0:
        for x in row.data_matches:
            ship_out, ship_return = str(x['ship_out']), str(x['ship_return']) 
            rank, place_of_origin = str(x['rank']), str(x['place_of_origin'])
            detected_ships = []
            detected_ranks = []
            detected_location = []
            
            # Find Ships
            if fuzz.partial_ratio(ship_out.lower(), str(row.beschrijving).lower()) >= 80 or fuzz.partial_ratio(ship_out.lower(), row.text.lower()) >= 80:
                    detected_ships.append(ship_out)
            if fuzz.partial_ratio(ship_return.lower(), str(row.beschrijving).lower()) >= 80 or fuzz.partial_ratio(ship_return.lower(), row.text.lower()) >= 80:
                    detected_ships.append(ship_return)

            # Find Rank
            if fuzz.partial_ratio(rank.lower(), str(row.beschrijving).lower()) >= 80 or fuzz.partial_ratio(rank.lower(), row.text.lower()) >= 80:
                detected_ranks.append(rank)

            # Find Place of Origin
            if fuzz.partial_ratio(place_of_origin.lower(), str(row.beschrijving).lower()) >= 80 or fuzz.partial_ratio(place_of_origin.lower(), row.text.lower()) >= 80:
                detected_location.append(place_of_origin)
            else:
                for ent in nlp(row.text).ents:
                    if ent.label_ == 'LOC':
                        if fuzz.ratio(place_of_origin, ent.text) >= 85:
                            detected_location.append(place_of_origin)
            holder[x['index']] = {'ships':detected_ships, 'rank':detected_ranks, 'location':detected_location}
        return holder
    else:
        return 0

def create_annotation_subset(notary, voc):
    nlp = spacy.load('nl_core_news_sm')
    return_df = notary.copy()
    return_df['index_matches'] = find_matches(return_df.name, voc, 80)
    return_df['data_matches'] = return_df.index_matches.progress_apply(get_voc_data, args=[voc])
    return_df = return_df.drop('index_matches', axis=1)
    return_df['data_entry'] = return_df.progress_apply(get_notary_matches, args=[nlp], axis=1)
    return return_df

# def annotate(row):
#     notary_date = datetime.strptime(row.datering, '%Y-%m-%d')
#     if row.data_matches != '0':
#         #print(row.data_matches)
#         #for person in row.data_matches:
#         for person in row.data_matches:
#             try:
#                 out_date = datetime.strptime(person['date_out'], '%Y-%m-%d')
#             except:
#                 out_date = datetime(year=1, month=1, day =1 )
#             try:
#                 return_date = datetime.strptime(person['date_return'], '%Y-%m-%d')
#             except:
#                 return_date = datetime(year=1, month=1, day =1 )
#             if (notary_date - out_date).days not in range(0, -91, -1) and (notary_date - return_date).days not in range(0, 91):
#                 #print('Skipped match')
#                 continue
                
#             else:
#                 print('{:10} | {:30} | {}'.format(' ', "Notary Information " + str(row.name), 'VOC Information ' + str(person['index'])))
#                 print('-' * 108)
#                 print('{:10} | {:30} | {} / {}'.format('Name', row['name'], person['name_original'], person['name_normalized']))
#                 print('{:10} | {:30} | {} / {}'.format('Dates', row.datering, person['date_out'], person['date_return']))
#                 print('{:10} | {:30} | {} / {}'.format('Ships', ' / '.join(row['data_entry'][str(person['index'])]['ships']), person['ship_out'], person['ship_return']))
#                 print('{:10} | {:30} | {}'.format('Rank', ' / '.join(row['data_entry'][str(person['index'])]['rank']), person['rank']))
#                 print('{:10} | {:30} | {}'.format('Locations', ' / '.join(row['data_entry'][str(person['index'])]['location']), person['place_of_origin']))
#                 check = False
#                 print('Are these persons the same? y/n:')
#                 while check != True:
#                     answer = input()
#                     if answer == 'y':
#                         return (person['name_original'], person['index'])
#                         check = True
#                     elif answer == 'n':
#                         check = True
#                     elif answer == 'text':
#                         print(row.text)
#                     else:
#                         print("Invalid input please enter 'y', 'n', or 'text' without the quotes.")
#         return None
                
#     else:
#         return None

In [5]:
def annotate(df, prev=None):
    if prev is not None:
        final = prev.copy()
        start = len(final)
    else:
        final = pd.DataFrame(columns = df.columns)
        start = 0
    holder = []
    #stop = False
    #while stop != True:
    for row in df[start:].itertuples():
        notary_date = datetime.strptime(row.datering, '%Y-%m-%d')
        if row.data_matches != '0':
            #print(row.data_matches)
            #for person in row.data_matches:
            for person in row.data_matches:
                try:
                    out_date = datetime.strptime(person['date_out'], '%Y-%m-%d')
                except:
                    out_date = datetime(year=1, month=1, day =1 )
                try:
                    return_date = datetime.strptime(person['date_return'], '%Y-%m-%d')
                except:
                    return_date = datetime(year=1, month=1, day =1 )
                if (notary_date - out_date).days not in range(0, -91, -1) and (notary_date - return_date).days not in range(0, 91):
                    #print('Skipped match')
                    continue

                else:
                    print('{:10} | {:30} | {}'.format(' ', "Notary Information " + str(row.name), 'VOC Information ' + str(person['index'])))
                    print('-' * 108)
                    print('{:10} | {:30} | {} / {}'.format('Name', row.name, person['name_original'], person['name_normalized']))
                    print('{:10} | {:30} | {} / {}'.format('Dates', row.datering, person['date_out'], person['date_return']))
                    print('{:10} | {:30} | {} / {}'.format('Ships', ' / '.join(row.data_entry[str(person['index'])]['ships']), person['ship_out'], person['ship_return']))
                    print('{:10} | {:30} | {}'.format('Rank', ' / '.join(row.data_entry[str(person['index'])]['rank']), person['rank']))
                    print('{:10} | {:30} | {}'.format('Locations', ' / '.join(row.data_entry[str(person['index'])]['location']), person['place_of_origin']))
                    check = False
                    print('Are these persons the same? y/n:')
                    while check != True:
                        answer = input()
                        if answer == 'y':
                            holder.append((person['name_original'], person['index']))
                            final = final.append(df.loc[row.Index])
                            check = True
                        elif answer == 'n':
                            check = True
                        elif answer == 'text':
                            print(row.text)
                        elif answer == 'stop':
                            final['vocop_match'][start:] = holder
                            return final
                        else:
                            print("Invalid input please enter 'y', 'n', 'stop', or 'text' without the quotes.")
                    if answer == 'y':
                        break
                    else:
                        continue
            if answer == 'y':
                continue
            else:
                holder.append(0)
                final = final.append(df.loc[row.Index])
        else:
            holder.append(0)
            final = final.append(df.loc[row.Index])
    final['vocop_match'][start:] = holder
    return final

In [2]:
#henk1 = create_annotation_subset(name_df[0:10].copy(), voc_df)
#henk1 = create_annotation_subset(name_df[0:5000].copy(), voc_df)
#henk1 = create_annotation_subset(name_df[54465:54467].copy(), voc_df)

In [11]:
#henk1.to_json('subset1.json')
#subset = pd.read_json('result.json')

In [3]:
#t5t10 = create_annotation_subset(name_df[5000:10000].copy(), voc_df)

In [4]:
#t5t10.to_json('subset2.json')

In [2]:
sub = pd.read_json('subset1.json')
sub2 = pd.read_json('subset2.json')

In [11]:
sub2[sub2.data_matches != 0][2450:]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry
9466,f6ba79ad-5509-50df-9bee-609ed8a5effb,358,JAN VERLEIJ,11917,16387,Kwitantie,1752-05-16,nederlands,"\nschip: Vredenrijk, kosten drank, kleding en ...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05616000065.JPG', 'KLAB05616000066.JPG',...",155No: 157 Quitantie gecasseerd den 16e: maij ...,Jochem Been,"[{'index': 16433, 'name_original': 'Jochem Kee...","{'16433': {'ships': [], 'rank': [], 'location'..."
9467,f6ba79ad-5509-50df-9bee-609ed8a5effb,358,JAN VERLEIJ,11917,16387,Kwitantie,1752-05-16,nederlands,"\nschip: Vredenrijk, kosten drank, kleding en ...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05616000065.JPG', 'KLAB05616000066.JPG',...",155No: 157 Quitantie gecasseerd den 16e: maij ...,Jan Meuleman,"[{'index': 2231, 'name_original': 'Jan Meulman...","{'2231': {'ships': [], 'rank': [], 'location':..."
9468,f6ba79ad-5509-50df-9bee-609ed8a5effb,358,JAN VERLEIJ,11917,16387,Kwitantie,1752-05-16,nederlands,"\nschip: Vredenrijk, kosten drank, kleding en ...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05616000065.JPG', 'KLAB05616000066.JPG',...",155No: 157 Quitantie gecasseerd den 16e: maij ...,Jan Laurense,"[{'index': 848, 'name_original': 'Jan Lourense...","{'848': {'ships': [], 'rank': [], 'location': ..."
9469,f6ba79ad-5509-50df-9bee-609ed8a5effb,358,JAN VERLEIJ,11917,16387,Kwitantie,1752-05-16,nederlands,"\nschip: Vredenrijk, kosten drank, kleding en ...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05616000065.JPG', 'KLAB05616000066.JPG',...",155No: 157 Quitantie gecasseerd den 16e: maij ...,Jacob Dreves,"[{'index': 360827, 'name_original': 'Jacob Dre...","{'360827': {'ships': [], 'rank': [], 'location..."
9470,f6ba79ad-5509-50df-9bee-609ed8a5effb,358,JAN VERLEIJ,11917,16387,Kwitantie,1752-05-16,nederlands,"\nschip: Vredenrijk, kosten drank, kleding en ...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05616000065.JPG', 'KLAB05616000066.JPG',...",155No: 157 Quitantie gecasseerd den 16e: maij ...,Melchior Muijssel,"[{'index': 472874, 'name_original': 'Melchior ...","{'472874': {'ships': [], 'rank': [], 'location..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,f31e7c18-1c99-65aa-27d3-803a7b9d8934,358,JAN VERLEIJ,11923,34080,Boedelinventaris,1754-06-11,nederlands,None,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05718000169.JPG', 'KLAB05718000170.JPG',...",No: 219 Specificatie gecasseerd den 18e: Janij...,Jan Potjes,"[{'index': 26488, 'name_original': 'Jan Potje'...","{'26488': {'ships': ['nan'], 'rank': [], 'loca..."
9993,f31e7c18-1c99-65aa-27d3-803a7b9d8934,358,JAN VERLEIJ,11923,34080,Boedelinventaris,1754-06-11,nederlands,None,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05718000169.JPG', 'KLAB05718000170.JPG',...",No: 219 Specificatie gecasseerd den 18e: Janij...,Wijbrand Wijbrandsz,"[{'index': 60137, 'name_original': 'Sijbrand W...","{'60137': {'ships': ['nan'], 'rank': [], 'loca..."
9995,f31e7c18-1c99-65aa-27d3-803a7b9d8934,358,JAN VERLEIJ,11923,34080,Boedelinventaris,1754-06-11,nederlands,None,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05718000169.JPG', 'KLAB05718000170.JPG',...",No: 219 Specificatie gecasseerd den 18e: Janij...,Hendrik Weeling,"[{'index': 28075, 'name_original': 'Hendrik Be...","{'28075': {'ships': ['nan'], 'rank': [], 'loca..."
9996,f31e7c18-1c99-65aa-27d3-803a7b9d8934,358,JAN VERLEIJ,11923,34080,Boedelinventaris,1754-06-11,nederlands,None,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05718000169.JPG', 'KLAB05718000170.JPG',...",No: 219 Specificatie gecasseerd den 18e: Janij...,Casper Harmensz,"[{'index': 255613, 'name_original': 'Casper Ha...","{'255613': {'ships': ['nan'], 'rank': [], 'loc..."


In [20]:
sub[0:661].to_json('batch1_patrick.json')
sub[661:1350].to_json('batch1_thom.json')
sub[1350:1924].to_json('batch1_chiel.json')

In [25]:
sub[1924:2529].to_json('batch2_patrick.json')
sub[2529:3134].to_json('batch2_thom.json')
sub[3134:3797].to_json('batch2_chiel.json')

In [45]:
sub[3793:4416].to_json('batch3_patrick.json')
sub[4416:5000].to_json('batch3_thom.json')
sub2[0:611].to_json('batch3_chiel.json')

In [12]:
#sub2[1226:1936].to_json('batch1.json')
sub2[3258:3844].to_json('batch2.json')
sub2[3844:4458].to_json('batch3.json')
sub2[3844:10000].to_json('batch4.json')

In [8]:
sub2[1226:1936].to_json('batch4_patrick.json')
sub2[1936:2598].to_json('batch5_patrick.json')
sub2[2598:3258].to_json('batch6_patrick.json')

In [6]:
voc_df = pd.read_csv('../vocop-clustered-new.csv', sep='	')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
